<h1> Project Overview </h1>
<p> from conversation 12/1/2016
Robert wants to see installations over time for PG&E, SCE, SDG&E
 <br/>- average size and number of installations 
 <br/>- vendors installation (pie chart) 
</p>
<p> Source: https://www.californiasolarstatistics.ca.gov/data_downloads/ </p>
<h2> Components/Timeline: </h2>
    <ul>
        <li> 1: Pull data (PYthon) and explration (Power Bi or Tabluae) for what will be analyzed. Determine what plots to use
            <ul>
                <li> Utility installations </li>
                <li> installers vs installation over time </li>
                <li> Could aslo look at installers with increasing rate of installation </li> 
                <li> Could also look at utility application time </li>
                <li> ... </li>
            </ul>
        </li>
        <li> 2: once we know what to do with the data, script in python to easily plot </li>
        <li> 3: incorporate automatic pull from the web to plot them. </li>
        <li> 4: Provide web/interactive plots </li>
    </ul>

<h1> 2. Run Analysis Script </h1>
<p> Run this script AFTER running "1_clean_and_filter" script, which creates the clean data set loaded into this script ("filtered_data.xlsx")</p>
<p> This will include:</p>
<ol>
    <li> Looking at the data and outputing dataframes for easy input into other visualization tools (like Power BI)</li>
    <li> Plotting the data in python and generating these graphs, saving them to a list of pngs [] </li>
    <li> Creating a results table (called 'res') for easy output to... (next step) </li>
    <li> Outputting to an html template </li>
</ol>

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import datetime as datetime
import scipy as scipy
from scipy.stats import mode
from bokeh.charts import Bar, output_file, show


In [2]:
# Create Dataframe to hold results:
res = pd.DataFrame(columns=['Result','Description'])
def addnewres(result, description,refreshRes=False):
    """Function for adding new insights or results to a dataframe to later iterate through. ONly takes 2 arguments, the result and description, can be any type of data."""
    if refreshRes==True:
        return pd.DataFrame(columns=['Result','Description'])
    else:
        dataApp = pd.DataFrame(columns=['Result','Description'])
        dataApp.set_value(0, 'Result', result)
        dataApp.set_value(0, 'Description', description)
        return res.append(dataApp).reset_index(drop=True)

<h2>1. Analysis of NEM Current Interconnections</h2>

In [3]:
#Load Data
filter_df = pd.read_excel('filtered_data.xlsx')

In [4]:
res = addnewres('','Removed top & bottom .5% of all systems based on size')

<h3> General code to look at data </h3>

<h3>How do utilities compare?</h3>

Seperate installs for each utility. System Size DC Capacity

In [5]:
"""
# [utility]_installs is the capacity in kW that was added for each day
SCE = filter_df[filter_df['Utility'] == 'SCE']
SCE_installs = SCE.loc[0:,['System Size DC','App Approved Date']]
SCE_installs = SCE_installs.groupby(SCE_installs['App Approved Date']).sum()
SCE_installs = SCE_installs.dropna()

PGE = filter_df[filter_df['Utility'] == 'PGE']
PGE_installs = PGE.loc[0:,['System Size DC','App Approved Date']]
PGE_installs = PGE_installs.groupby('App Approved Date').sum()
PGE_installs = PGE_installs.dropna()

SDGE = filter_df[filter_df['Utility'] == 'SDGE']
SDGE_installs = SDGE.loc[0:,['System Size DC','App Approved Date']]
SDGE_installs = SDGE_installs.groupby('App Approved Date').sum()
SDGE_installs = SDGE_installs.dropna()
"""

"\n# [utility]_installs is the capacity in kW that was added for each day\nSCE = filter_df[filter_df['Utility'] == 'SCE']\nSCE_installs = SCE.loc[0:,['System Size DC','App Approved Date']]\nSCE_installs = SCE_installs.groupby(SCE_installs['App Approved Date']).sum()\nSCE_installs = SCE_installs.dropna()\n\nPGE = filter_df[filter_df['Utility'] == 'PGE']\nPGE_installs = PGE.loc[0:,['System Size DC','App Approved Date']]\nPGE_installs = PGE_installs.groupby('App Approved Date').sum()\nPGE_installs = PGE_installs.dropna()\n\nSDGE = filter_df[filter_df['Utility'] == 'SDGE']\nSDGE_installs = SDGE.loc[0:,['System Size DC','App Approved Date']]\nSDGE_installs = SDGE_installs.groupby('App Approved Date').sum()\nSDGE_installs = SDGE_installs.dropna()\n"

Aggregate over months, years, and quarters

In [ ]:
"""
#aggregate monthly
SCE_installs_monthly = SCE_installs.resample('M').sum()
PGE_installs_monthly = PGE_installs.resample('M').sum()
SDGE_installs_monthly = SDGE_installs.resample('M').sum()

#aggregate yearly (sum at year end)
SCE_installs_yearly = SCE_installs.resample('A').sum()
PGE_installs_yearly = PGE_installs.resample('A').sum()
SDGE_installs_yearly = SDGE_installs.resample('A').sum()

#aggregate quarterly (sum at year end)
SCE_installs_qly = SCE_installs.resample('3M').sum()
PGE_installs_qly = PGE_installs.resample('3M').sum()
SDGE_installs_qly = SDGE_installs.resample('3M').sum()

#aggregate yearly installations (count not sum)
SCE_installs_count_yearly = SCE_installs.resample('A').count()
PGE_installs_count_yearly = PGE_installs.resample('A').count()
SDGE_installs_count_yearly = SDGE_installs.resample('A').count()
"""

In [6]:
#then record the latest App Approval date for results summary table to output to HTML
date_S = pd.to_datetime(filter_df['App Approved Date'])
res = addnewres(date_S.max().date(),'Last date recorded in data')
del date_S #to free up space

<h4>Output a dataframe of the cummulative sum of system size DC (kw)</h4>

In [7]:
#get 4 columns seperately then merge in a dataframe cumm_I
cols = ['System Size DC', 'Utility','App Approved Date']
sel = filter_df[cols]
sel = sel.dropna(axis=0,how='any')

total_I = sel.groupby('App Approved Date').sum()
total_I = total_I.cumsum()
total_I = total_I/1000
total_I = total_I.astype(int)

SCE_I = sel[sel['Utility']=='SCE'].groupby('App Approved Date').sum()
SCE_I = SCE_I.cumsum()
SCE_I.name = 'SCE'
SCE_I = SCE_I/1000
SCE_I = SCE_I.astype(int)

PGE_I = sel[sel['Utility']=='PGE'].groupby('App Approved Date').sum()
PGE_I = PGE_I.cumsum()
PGE_I.name = 'PGE'
PGE_I = PGE_I/1000
PGE_I = PGE_I.astype(int)

SDGE_I = sel[sel['Utility']=='SDGE'].groupby('App Approved Date').sum()
SDGE_I = SDGE_I.cumsum()
SDGE_I.name = 'SDGE'
SDGE_I = SDGE_I/1000
SDGE_I = SDGE_I.astype(int)

cumm_I = pd.DataFrame(index=total_I.index)

cumm_I['Total'] = total_I

cumm_I = cumm_I.join(PGE_I)
cumm_I.columns = ['Total','PGE']

cumm_I = cumm_I.join(SCE_I)
cumm_I.columns = ['Total','PGE','SCE']


cumm_I = cumm_I.join(SDGE_I)
cumm_I.columns = ['Total','PGE','SCE','SDGE']

"""
cumm_I.plot(title='All Installations cummulative', legend=True)
cumm_I[cumm_I.index.year>=2014].plot(title='Installations cummulative after 2013', legend=True)
"""

cumm_I.to_excel('output_dfs\Cummulative Sum of System Size By Utility.xlsx')

<h4> Output dataframe grouped by location (zip) and total installs (sum) and distinct </h4>

In [8]:
grpcity = filter_df.groupby('Service City')['System Size DC']
col1 = grpcity.sum()
col1.name='Sum Syst Size kW'
col2 = grpcity.mean()
col2.name='Avg Syst Size kW'
col3 = grpcity.count()
col3.name='Number of Installs'

In [9]:
grpcity = pd.concat([col1,col2,col3],axis=1,join='inner')

In [10]:
grpcity.to_excel('output_dfs/installations by city.xlsx')

<h3>Vendor Breakdown: </h3>

In [11]:
# Find the top 10 vendor performance (# of installs and total install capacity) for each utilty
vendor_sum = pd.DataFrame()

for util in ['SCE','PGE','SDGE']:
    col1 = filter_df[filter_df['Utility']==util].groupby('Installer Name')['System Size DC'].count().sort_values(ascending=False).head(10)
    col1.name="Number of Installs"

    col2 = filter_df[filter_df['Utility']==util].groupby('Installer Name')['System Size DC'].sum().sort_values(ascending=False).head(10)/1000
    col2.name="Total Capacity Installed (MW)"

    col3 = pd.Series( [util]*len(col2))
    col3.index = col2.index
    col3.name = "Utility"
    
    vdf = pd.concat([col1,col2,col3],axis=1,join='outer')
    vendor_sum = vendor_sum.append( vdf )

vendor_sum.to_excel('output_dfs/top vendors by utility.xlsx')

In [12]:
# Find the top 15 vendors in california
col1 = filter_df.groupby('Installer Name')['System Size DC'].count().sort_values(ascending=False)/1000
col1.name="Number of Installs"
col2 = filter_df.groupby('Installer Name')['System Size DC'].sum().sort_values(ascending=False)/1000
col2.name="Total Capacity Installed (MW)"

topvendors = pd.concat([col1,col2],axis=1,join='outer').sort_values(['Total Capacity Installed (MW)','Number of Installs'],ascending=False).head(15)

topvendors.to_excel('output_dfs/top vendors.xlsx')

In [25]:
SCE_app_time = pd.to_datetime(filter_df['App Complete Date'][filter_df['Utility']=='SCE']) - pd.to_datetime(filter_df['App Received Date'][filter_df['Utility']=='SCE'])
PGE_app_time = pd.to_datetime(filter_df['App Complete Date'][filter_df['Utility']=='PGE']) - pd.to_datetime(filter_df['App Received Date'][filter_df['Utility']=='PGE'])
SDGE_app_time = pd.to_datetime(filter_df['App Complete Date'][filter_df['Utility']=='SDGE']) - pd.to_datetime(filter_df['App Received Date'][filter_df['Utility']=='SDGE'])


<h2>Plot results </h2>

Plot nicely:

In [ ]:
pngs=[]
pngscount=0

In [ ]:
# These are the "Tableau 20" colors as RGB. 

colorbrew = ['#7b3294','#5ab4ac','#A1ABA5','#d8b365','#c2a5cf','#a6dba0','#008837']

tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    

# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)    

#Change face color
pyplot.rcParams['axes.facecolor'] = 'white'
    
# You typically want your plot to be ~1.33x wider than tall.     
# Common sizes: (10, 7.5) and (12, 9)    
pyplot.figure(figsize=((6*1.33),6))  

# Remove the plot frame lines. They are unnecessary chartjunk.    
ax = pyplot.subplot(111)    
ax.spines["top"].set_visible(False)    
ax.spines["bottom"].set_visible(False)    
ax.spines["right"].set_visible(False)    
ax.spines["left"].set_visible(False)    
  
# Ensure that the axis ticks only show up on the bottom and left of the plot.    
# Ticks on the right and top of the plot are generally unnecessary chartjunk.    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

# Limit the range of the plot to only where the data is.    
# Avoid unnecessary whitespace.    
pyplot.ylim(0, cumm_I['Total'].max())    
pyplot.xlim(cumm_I.index.date.min(), cumm_I.index.date.max())  

  
# Make sure your axis ticks are large enough to be easily read.    
# You don't want your viewers squinting to read your plot.    
pyplot.yticks(fontsize=14)    
pyplot.xticks(fontsize=14)    

"""
# Provide tick lines across the plot to help your viewers trace along    
# the axis ticks. Make sure that the lines are light and small so they    
# don't obscure the primary data lines.    
for y in range(10, 91, 10):    
    plt.plot(range(1968, 2012), [y] * len(range(1968, 2012)), "--", lw=0.5, color="black", alpha=0.3)    
""" 
# Remove the tick marks; they are unnecessary with the tick lines we just plotted.    
pyplot.tick_params(axis="both", which="both", bottom="off", top="off",    
                labelbottom="on", left="off", right="off", labelleft="on")    
  
# Now that the plot is prepared, it's time to actually plot the data!    
#     
majors = cumm_I.columns

for rank, column in enumerate(majors):    
    # Plot each line separately with its own color, using the Tableau 20    
    # color set in order.    
    pyplot.plot(cumm_I.index.date,    
            cumm_I[column].values,    
            lw=4, color=colorbrew[rank])    
  
    # Add a text label to the right end of every line. Most of the code below    
    # is adding specific offsets y position because some labels overlapped.    
    y_pos = cumm_I[column].values[-1] - 0.5    
    if column == "Total":    
        y_pos += 0.5    
    elif column == "SCE":    
        y_pos -= 0.5    
    elif column == "PGE":    
        y_pos += 0.75    
    elif column == "SDGE":    
        y_pos -= 0.25    
  
  
    # Again, make sure that all labels are large enough to be easily read    
    # by the viewer.    
    #pyplot.text(cumm_I.index.date.max(), y_pos, column, fontsize=14, color=tableau20[rank])    
  
# matplotlib's title() call centers the title on the plot, but not the graph,    
# so I used the text() call to customize where the title goes.    
  
# Make the title big enough so it spans the entire plot, but don't make it    
# so big that it requires two lines to show.    
  
# Note that if the title is descriptive enough, it is unnecessary to include    
# axis labels; they are self-evident, in this plot's case.  

pyplot.title("Solar PV installation capacity in California is increasing greatly over time"    
      , fontsize=17, ha="center")    
pyplot.ylabel("Installed Capacity MW", fontsize=12) 
pyplot.legend(list(cumm_I.columns),loc = 'upper left',fontsize = 14)
pngs.append("..//imgs//cummulative_PV_capacity_installed.png")

# Always include your data source(s) and copyright notice! And for your    
# data sources, tell your viewers exactly where the data came from,    
# preferably with a direct link to the data. Just telling your viewers    
# that you used data from the "U.S. Census Bureau" is completely useless:    
# the U.S. Census Bureau provides all kinds of data, so how are your    
# viewers supposed to know which data set you used?    
#pyplot.text(cumm_I.index.date.min(),  "Data source: ", fontsize=10)    
  
# Finally, save the figure as a PNG.    
# You can also save it as a PDF, JPEG, etc.    
# Just change the file extension in this call.    
# bbox_inches="tight" removes all the extra whitespace on the edges of your plot.    
pyplot.savefig(pngs[pngscount], bbox_inches="tight")  

In [ ]:
# Now that the plot is prepared, it's time to actually plot the data!    
#     
majors = cumm_I.columns
pyplot.figure(figsize=((6*1.33),6))  
for rank, column in enumerate(majors):    
    # Plot each line separately with its own color, using the Tableau 20    
    # color set in order.    
    pyplot.plot(cumm_I[cumm_I.index.year>=2014].index.date,    
            cumm_I[cumm_I.index.year>=2014][column].values,    
            lw=4, color=colorbrew[rank])    
  
    # Add a text label to the right end of every line. Most of the code below    
    # is adding specific offsets y position because some labels overlapped.    
    y_pos = cumm_I[column].values[-1] - 0.5    
    if column == "Total":    
        y_pos += 0.5    
    elif column == "SCE":    
        y_pos -= 0.5    
    elif column == "PGE":    
        y_pos += 0.75    
    elif column == "SDGE":    
        y_pos -= 0.25    
  
  
    # Again, make sure that all labels are large enough to be easily read    
    # by the viewer.    
    #pyplot.text(cumm_I.index.date.max(), y_pos, column, fontsize=14, color=tableau20[rank])    
  
# matplotlib's title() call centers the title on the plot, but not the graph,    
# so I used the text() call to customize where the title goes.    
  
# Make the title big enough so it spans the entire plot, but don't make it    
# so big that it requires two lines to show.    
  
# Note that if the title is descriptive enough, it is unnecessary to include    
# axis labels; they are self-evident, in this plot's case.    
pyplot.title("A closer look shows that this adoption could be slowing"    
      , fontsize=17, ha="center")    
pyplot.ylabel("Installed Capacity MW", fontsize=12) 
pyplot.legend(list(cumm_I.columns),loc = 'upper left',fontsize = 14)

# Always include your data source(s) and copyright notice! And for your    
# data sources, tell your viewers exactly where the data came from,    
# preferably with a direct link to the data. Just telling your viewers    
# that you used data from the "U.S. Census Bureau" is completely useless:    
# the U.S. Census Bureau provides all kinds of data, so how are your    
# viewers supposed to know which data set you used?    
#pyplot.text(cumm_I.index.date.min(),  "Data source: ", fontsize=10)    
  
# Finally, save the figure as a PNG.    
# You can also save it as a PDF, JPEG, etc.    
# Just change the file extension in this call.    
# bbox_inches="tight" removes all the extra whitespace on the edges of your plot. 

pngs.append("..//imgs//cummulative_PV_capacity_installed_zoom.png")
pngscount+=1
pyplot.savefig(pngs[pngscount], bbox_inches="tight")  


In [ ]:
pyplot.figure(figsize=((6*1.33),6))
pyplot.plot(SCE_installs_monthly.index, SCE_installs_monthly['System Size DC'], linewidth=4, color=colorbrew[1], label='SCE')
pyplot.plot(PGE_installs_monthly.index, PGE_installs_monthly['System Size DC'], linewidth=4, color=colorbrew[2], label='PGE')
pyplot.plot(SDGE_installs_monthly.index, SDGE_installs_monthly['System Size DC'], linewidth=4, color=colorbrew[3], label='SDGE')
pyplot.xlabel('Date',fontsize=14)
pyplot.ylabel('Capacity kW',fontsize=14)
pyplot.title('New Monthly Installed Capacity Added',fontsize=14)
pyplot.legend(loc = 'upper left')


pngs.append("..//imgs//monthly_install_cap_agg.png")
pngscount+=1
pyplot.savefig(pngs[pngscount], bbox_inches="tight")   

pyplot.figure(figsize=((6*1.33),6))
#aggregate monthly - Installs under 1MW
SCE_installs_monthly = SCE_installs[SCE_installs['System Size DC']<1000].resample('M').sum()
PGE_installs_monthly = PGE_installs[PGE_installs['System Size DC']<1000].resample('M').sum()
SDGE_installs_monthly = SDGE_installs[SDGE_installs['System Size DC']<1000].resample('M').sum()

pyplot.figure(figsize=((6*1.33),6))
pyplot.plot(SCE_installs_monthly.index, SCE_installs_monthly['System Size DC'], linewidth=4, color=colorbrew[1], label='SCE') #green
pyplot.plot(PGE_installs_monthly.index, PGE_installs_monthly['System Size DC'], linewidth=4, color=colorbrew[2], label='PGE') #black
pyplot.plot(SDGE_installs_monthly.index, SDGE_installs_monthly['System Size DC'], linewidth=4, color=colorbrew[3], label='SDGE') #salmon
pyplot.xlabel('Date')
pyplot.ylabel('Monthly New Installed Capacity')
pyplot.title('Capacity Added Monthly - Installations < 1MW')
pyplot.legend()

pngs.append("..//imgs//monthly_install_cap_agg_lessthan_1MW.png")
pngscount+=1
pyplot.savefig(pngs[pngscount], bbox_inches="tight")   

<h2>Some Stats</h2>

In [ ]:
m = mode(filter_df['System Size DC'])[0]
res = addnewres(m,'Most common System Size installed (Mode)')


filter_df['System Size DC'].describe()

res = addnewres(len(filter_df),'Total completed Applications Reported')
res = addnewres(filter_df['System Size DC'].max(),'Biggest Installation Size')
res = addnewres(filter_df['System Size DC'][filter_df['System Size DC']>0].min(),'Smallest Installation Size')

res

In [ ]:
filter_df['System Size DC'][(filter_df['System Size DC']<20) & (filter_df['System Size DC']>0)].hist(bins=20,color = '#f5f5f5')
pyplot.title('Histogram of solar installations by kW (showing only systems <20kW)',fontsize=14)
pngs.append('..//imgs//hist_installs_size.png')
pngscount+=1
pyplot.savefig(pngs[pngscount], bbox_inches="tight")   
res = addnewres(str(filter_df['System Size DC'][(filter_df['System Size DC']<20) & (filter_df['System Size DC']>0)].count()*100/len(filter_df['System Size DC']))[:4]+'%','Most Sytems installed are less than 20 kW')

<h3>Vendor breakdown</h3>

Top 20 Installers:

In [ ]:
vendors_top20 = filter_df['Installer Name'].value_counts().head(20)
vendors_top10_SCE = filter_df['Installer Name'][filter_df['Utility']=='SCE'].value_counts().head(10)
vendors_top10_PGE = filter_df['Installer Name'][filter_df['Utility']=='PGE'].value_counts().head(10)
vendors_top10_SDGE = filter_df['Installer Name'][filter_df['Utility']=='SDGE'].value_counts().head(10)


In [ ]:
vendors_top20.plot(kind='barh',rot=0, fontsize=11, color = '#f5f5f5',title = 'Top 20 vendors in California  by # of installations')
pngs.append("..//imgs//hist_vendors_top20.png")
pngscount+=1
pyplot.savefig(pngs[pngscount], bbox_inches="tight")   

<h4>Installers by Utility</h4>

In [ ]:
vendors_top10_SCE.plot(kind='barh',rot=0, color='#5ab4ac')
vendors_top10_PGE.plot(kind='barh',rot=0, color='#f5f5f5')
vendors_top10_SDGE.plot(kind='barh',rot=0, color='#d8b365')
pyplot.xlabel('# of PV Installations Completed')
pyplot.title("The top 10 installers in each IOU territory varies")
pyplot.legend(('SCE','PGE','SDGE'),fontsize=14)
pngs.append("..//imgs//IOUs_top10_vendors.png")
pngscount+=1
pyplot.savefig(pngs[3], bbox_inches="tight") 

In [ ]:
fig, (ax1,ax2,ax3) = pyplot.subplots(1,3, figsize=(14, 4))
    
top5_SCE = filter_df['Installer Name'][filter_df['Utility']=='SCE'].value_counts().head(5)
top5_PGE = filter_df['Installer Name'][filter_df['Utility']=='PGE'].value_counts().head(5)
top5_SDGE = filter_df['Installer Name'][filter_df['Utility']=='SDGE'].value_counts().head(5)

top5_SCE.plot(title='SCE',ax=ax1,kind='barh',rot=0, color='#5ab4ac')
top5_PGE.plot(title='PGE',ax=ax3,kind='barh',rot=0, color='#f5f5f5')
top5_SDGE.plot(title='SDGE',ax=ax2,kind='barh',rot=0, color='#d8b365')

ax2.set_xlabel('# of PV Installations Completed', fontsize=12)

ax1.set_title("SCE")
ax3.set_title("PGE")
ax2.set_title("SDGE")

#fig.text(0,1,'Say something if you wnat here')

xmax = max( (top5_SCE.max(),top5_PGE.max(),top5_SDGE.max()) )

ax1.set_xlim(0,xmax)
ax2.set_xlim(0,xmax)
ax3.set_xlim(0,xmax)

pngs.append("..//imgs//IOUs_top5_vendors.png")
pngscount+=1
pyplot.savefig(pngs[3], bbox_inches="tight")  


<h2> Create Report in HTML</h>

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open("..//report_template.html"),"html")

results = res.to_html(bold_rows=False, index=False).replace('border="1"','border="0"')
add_soup = BeautifulSoup(results)

soup.table.contents = add_soup.table

for png in pngs:
    new_tag = soup.new_tag("img", src=('imgs'+png[9:]))
    soup.find("div",{'id':'plotsdiv'}).append((new_tag))
    soup.find("div",{'id':'plotsdiv'}).append(soup.new_tag("br"))
    soup.find("div",{'id':'plotsdiv'}).append(soup.new_tag("br"))


html = soup.prettify("utf-8")
with open("..//index.html", "wb") as file:
    file.write(html)

In [ ]:
cumm_I['SCE'].max()
